In [ ]:
import numpy as np
from scipy.linalg import fractional_matrix_power

In [ ]:
def compute_covariance_matrices(X, Y):
    """
    Computes the covariance matrices C00, C0τ, and Cττ.

    Args:
        X (numpy.ndarray): Matrix of shape (d, N) representing past states.
        Y (numpy.ndarray): Matrix of shape (d, N) representing future states.

    Returns:
        C00, C0τ, Cττ: Covariance matrices.
    """
    T_tau = X.shape[1]  # Number of time steps (T - τ)

    C00 = (X @ X.T) / T_tau
    C0tau = (X @ Y.T) / T_tau
    Ctautau = (Y @ Y.T) / T_tau

    return C00, C0tau, Ctautau

def compute_whitened_Koopman(C00, C0tau, Ctautau):
    """
    Computes the whitened Koopman matrix.

    Args:
        C00 (numpy.ndarray): Covariance matrix of X.
        C0tau (numpy.ndarray): Cross-covariance matrix of X and Y.
        Ctautau (numpy.ndarray): Covariance matrix of Y.

    Returns:
        Koopman matrix K_tilde.
    """
    C00_inv_sqrt = fractional_matrix_power(C00, -0.5)
    Ctautau_inv_sqrt = fractional_matrix_power(Ctautau, -0.5)

    K_tilde = C00_inv_sqrt @ C0tau @ Ctautau_inv_sqrt
    return K_tilde

def compute_svd_rank_d(K_tilde, d):
    """
    Computes the rank-d approximation of the Koopman matrix using SVD.

    Args:
        K_tilde (numpy.ndarray): The whitened Koopman matrix.
        d (int): Target dimensionality.

    Returns:
        Encoding matrix E, decoding matrix D.
    """
    U, Sigma, Vt = np.linalg.svd(K_tilde, full_matrices=False)

    # Take the first d singular values/vectors
    U_d = U[:, :d]
    Sigma_d = np.diag(Sigma[:d])
    V_d = Vt[:d, :]

    return U_d, Sigma_d, V_d

# Example Usage
T = 754  # Total time series length
d = 10   # Dimension of the state space
tau = 5  # Time lag

# Generate random time series data for X and Y
X = np.random.randn(d, T - tau)  # Shape (d, T-tau)
Y = np.random.randn(d, T - tau)  # Shape (d, T-tau)

# Step 1: Compute covariance matrices
C00, C0tau, Ctautau = compute_covariance_matrices(X, Y)

# Step 2: Compute whitened Koopman matrix
K_tilde = compute_whitened_Koopman(C00, C0tau, Ctautau)

# Step 3: Compute SVD and rank-d approximation
U_d, Sigma_d, V_d = compute_svd_rank_d(K_tilde, d)

# Compute final encoding and decoding matrices
E = Sigma_d @ U_d.T @ fractional_matrix_power(C00, -0.5)
D = fractional_matrix_power(Ctautau, 0.5) @ V_d

print("Encoding Matrix E:", E.shape)
print("Decoding Matrix D:", D.shape)
